In [27]:
import torch
import os 
import sys
import yaml
import numpy as np
from typing import Optional, Union
from tqdm import tqdm

def path_link(path:str):
    sys.path.append(path)

path_link('/master/code/lib')

import features as ft
import simulation_v2 as sim

import NNSimulator as NNSim
from torch_geometric.data import Data


In [2]:
class Parameters():
    def __init__(self):
        self.dt = 0.001
        self.v0 = 60
        self.k = 70
        self.epsilon = 0.5
        self.tau = 3.5
        self.R = 1
        self.N = 200
        self.boundary = 100

        self.nbStep = 150

In [4]:
PATH = '/master/code/analyze_models/exps/exp-test'
MODEL_PATH = '/master/code/models/mod_base'

DISPLAY_PATH = ['/master/code/analyze_models/display/test']

In [6]:
def findModels(path):
    pathLists = []
    for root, dirs, files in tqdm(os.walk(path)):
            for file in files:
                  
                  if file.endswith('.pt'):
                        pathLists.append(os.path.join(root, file))


    return pathLists


def delete_wandb_dirs(start_path):
    for root, dirs, files in os.walk(start_path, topdown=False):
        for dir_name in dirs:
            if dir_name == "wandb":
                dir_path = os.path.join(root, dir_name)
                print(f"Deleting: {dir_path}")
                shutil.rmtree(dir_path)



def getName(path):
    run_name = path.split('/')[-3]

    model_type = path.split('/')[-1].split('.')[0]

    if 'best' in model_type:
        model_type = '_best'

    else:
         model_type = '_latest'

    name = run_name + model_type

    return name


def loadModel(modelName:str, inputShape:int = 8, edges_shape = 5, path = None):
    """ 
    Function to import the model

    Args:
    -----
        - `modelName`: name of the model
        - `inputShape`: inout shape of the NN
        - `edges_shape`: edge shape of the NN
        - `path`: path where the models are
    """

    sys.path.append(path)

    loadFun = __import__(f'{modelName}', fromlist = ('loadNetwork'))

    model = loadFun.loadNetwork(inputShape, edges_shape)

    return model



def getModelName(key):

    name = ''

    #if 'simplest' in key:
    #    name = name + 'simplest'
    
    name = name + 'simplest'

    ## other possibilities

    if 'no-dropout' in key:
        name = name + '_no-dropout'
    else:
        name = name + '_dropout'

    if 'no-encoder' in key:
        name = name + '_no-encoder'
    else:
        name = name + '_encoder'

    if 'relu' in key:
        name = name + '-relu'

    return name

In [13]:
def getData(params):
    N = params.N
    v0 = params.v0
    k = params.k
    eps = params.epsilon
    tau = params.tau
    R = params.R
    dt = params.dt
    nbStep = params.nbStep
    boundary = params.boundary

    lim = 0.85 * 100

    xPos = np.linspace(-lim, lim, 10)
    yPos = np.linspace(-lim, lim, 10)
    gridX, gridY = np.meshgrid(xPos, yPos)
    delta = np.random.uniform(0, 7, gridX.shape + (2,))

    gridX2 = gridX + delta[:, :, 0]
    gridY2 = gridY + delta[:, :, 1]

    pos = np.stack([gridX.ravel(), gridY.ravel()], axis=1)
    pos_perturbed = np.stack([gridX2.ravel(), gridY2.ravel()], axis=1)

    pos = np.concatenate([pos, pos_perturbed], axis=0)

    angles = np.random.rand(pos.shape[0]) * 2 * np.pi

    data = sim.compute_main(N, (v0, tau, k, eps), boundary, T = nbStep, initialization = (pos, angles), dt = dt)[0]

    x, y, attr, inds = ft.processSimulation(data)

    dataList = []

    for i in range(len(x)):
        g = Data(x = x[i][:, 2:], y = y[i], edge_attr = attr[i], edge_index = inds[i])
        dataList.append(g)

    return data, dataList, x

In [36]:
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


def simulate(modelList, T, tau = 8):
    params = Parameters()
    params.nbStep = T + tau

    with torch.no_grad():
        # get the data for the different models

        for model_path in tqdm(modelList):

            name_plot = getName(model_path)

            model = loadModel(getModelName(name_plot), path=MODEL_PATH)
            print(name_plot)
            print(model_path)
            print(getModelName(name_plot))
            std_dict = torch.load(model_path)
            model.load_state_dict(std_dict)
            model = model.to(DEVICE)
            model.eval()


            path = os.path.join(DISPLAY_PATH[0], f'{name_plot}.mp4')


            data, graphs, x = getData(params)

            startState = graphs[tau].to(DEVICE)

            

            out = NNSim.genSim(model, T, startState, x[tau][:, :2], train = False)

            return data, out


            







            

In [37]:
list_exp = ['/master/code/analyze_models/exps/exp-test']
exp = list_exp[0]
disp = DISPLAY_PATH[0]

model_list = findModels(exp)

4it [00:00, 40.25it/s]


In [38]:
print(len(model_list))

2


In [52]:
data, out = simulate(model_list, T = 150, tau = 8)

  0%|          | 0/2 [00:00<?, ?it/s]

>>>> loading simplest
INFO >>> with NO encoder
INFO >>> with NO dropout
master-thesis_normal_classic_dt-0.01_lr-0.001_batch-16_encoder-no-encoder_dropout-no-dropout-exp_best
/master/code/analyze_models/exps/exp-test/test_new_activation/master-thesis_normal_classic_dt-0.01_lr-0.001_batch-16_encoder-no-encoder_dropout-no-dropout-exp/model_trained/simplest_no-dropout_no-encoder_aug_best_0-001-test.pt
simplest_no-dropout_no-encoder
v0:60, tau:3.5, k:70, epsilon:0.5


  0%|          | 0/2 [00:36<?, ?it/s]


In [53]:
print(data[7:].shape)

(151, 200, 2)


In [54]:
print(out.shape)

torch.Size([151, 200, 2])


In [55]:
import display as disp

In [56]:
path = os.path.join(DISPLAY_PATH[0], f'test.mp4')
vidParams = disp.videoParameters(path)

In [57]:
d_gt = torch.from_numpy(data[7:])
d_repd = out

In [58]:

disp.compareVideo(d_repd.clone(), d_gt.clone(), vidParams,bounds=((-140, 140), (-140, 140)))

In [50]:
print(path)

/master/code/analyze_models/display/test/test.mp4
